In [1]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import requests
import numpy as np
import pandas as pd
import json

from bs4 import BeautifulSoup


In [4]:
#file1 = open('cards-standard.txt', 'r') 
#Lines = file1.readlines() 
#maximum = len(Lines)
#cards_key = {}
#cards_value = {}
#i=1
#for line in Lines:
 #   cards_value[i] = line.rstrip("\n")
  #  cards_key[line.rstrip("\n")] = i
   # i += 1

#print(cards_key["Arcane Breath"])
#print(cards_value[32])
#print(len(Lines))

In [3]:
classes_id = {'Demon Hunter' : 16384,
          'Druid' : 4,
          'Hunter' : 8,
          'Mage' : 16,
          'Paladin' : 32,
          'Priest' : 64, 
          'Rogue' : 128,
          'Shaman' : 256,
          'Warlock' : 512,
          'Warrior' : 1024}

r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id['Demon Hunter']))
soup = BeautifulSoup(r.content,"html.parser")
rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
links = rows.find_all('li')
print(links[len(links)-2].text)


6


In [4]:
#gets number of pages 
def page_max(deck_type):
    #request html from url
    r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id[deck_type]))
    soup = BeautifulSoup(r.content,"html.parser")
    #parse through html
    rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
    links = rows.find_all('li')
    return int(links[len(links)-2].text)

def scrape_decks(deck_type):
    start_urls = [] 
    #get number of pages
    n = page_max(deck_type)
    print('Number of Pages: ',n)
    #create the initial urls for deck_type
    for i in range(n):
        start_urls.append("https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=" + str(i+1)+"&filter-class="+ str(classes_id[deck_type]))
    deck_urls = []
    deck_classes = []
    #get every deck link on each page deck_type
    for url in start_urls:
        print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser") 
        rows = soup.find_all('table', class_="listing listing-decks b-table b-table-a")
        for row in rows:
            decks = row.find_all('span', class_="tip")
            for deck in decks:
                links = deck.find_all('a', href=True)
                for link in links:
                    deck_urls.append("https://www.hearthpwn.com" + link['href'])
                    
            deck_class = row.find_all('td', class_="col-class")
            for cl in deck_class:
                deck_classes.append(cl.text)
    #call function that gets card data within the collected urls
    return cards_from_url(deck_urls), deck_classes



In [5]:
#scrape cards in deck from separate url
def cards_from_url(deck_urls):
    decks = []
    for url in deck_urls:
        print(url)
        deck=[]
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        table = soup.find_all('table', class_="listing listing-cards-tabular b-table b-table-a")
        #navigating the page table to get card names
        for rows in table:
            row = rows.find_all('td', class_="col-name")
            for i in row:
                card = i.find('a')
                name = card.text
                number = card['data-count']
                #remove "\n" for later functions
                for num in range(int(number)):
                    deck.append(name.replace('\n',''))
        decks.append(deck)
    return decks


        


In [6]:
def deck_to_vector(deck):
    deck_vector = []
    for elem in deck:
        #remove any potential white spaces from strings
        #if type(deck) is not list:
        elem = elem.lstrip()
        elem = elem.rstrip()

        deck_vector.append(cards_key[elem])
    return deck_vector

def vector_to_deck(deck_vector):
    deck = []
    for elem in deck_vector:
        deck.append(cards_value[elem])
    return deck

def vectorize_deck_list(decks):
    vecks = []
    for d in decks:
        vecks.append(deck_to_vector(d))
    return vecks



In [9]:
#deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
#deck_type = ['Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock']
#deck_type = ['Paladin','Priest','Rogue','Shaman','Warlock']

#for cl in deck_type:
 #   print("Scraping information: ", cl)
  #  decks, deck_classes = scrape_decks(cl)
   # print("Creating ", cl ," decks")
    #collected_decks = concatenate_deck_class(decks, deck_classes)
    #collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks), deck_classes)
    #print("Creating ", cl, ".csv")
    #pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
    #pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+cl+".csv",index=False)



In [11]:
deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
#read json card data
file = open("data.json", "r")
cards = json.load(file)

to_convert = []
#for every class
for cl in deck_type:
    print("Scraping information: ", cl)
    #scrape data
    decks, deck_classes = scrape_decks(cl)
    print("Appending ", cl, " data")
    #append to array for conversion
    to_convert.append(decks)


Scraping information:  Mage
Number of Pages:  13
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=1&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=2&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=3&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=4&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=5&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=6&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=7&filter-class=16
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-

https://www.hearthpwn.com/decks/1383501-pingmage
https://www.hearthpwn.com/decks/1383491-hero-power-freeze-and-other-surprises
https://www.hearthpwn.com/decks/1383474-new-hero-power-mage-deck-is-a-blast-really
https://www.hearthpwn.com/decks/1383435-fb-yogg-mage
https://www.hearthpwn.com/decks/1383434-lunacy-cthun-spell-mage
https://www.hearthpwn.com/decks/1383368-grinder-hero-power-mage
https://www.hearthpwn.com/decks/1383336-spell-mage
https://www.hearthpwn.com/decks/1383335-dogs-spell-only
https://www.hearthpwn.com/decks/1383327-mordresh-king-of-the-outposts-tempo-hero-power
https://www.hearthpwn.com/decks/1383321-elemental-mage
https://www.hearthpwn.com/decks/1383312-hero-power-spell-mage
https://www.hearthpwn.com/decks/1383305-spell-mage
https://www.hearthpwn.com/decks/1383300-hero-power-mage
https://www.hearthpwn.com/decks/1383251-spell-damage-mage-freeze
https://www.hearthpwn.com/decks/1383234-ping-mage
https://www.hearthpwn.com/decks/1383233-garbage-mage
https://www.hearthpwn.c

https://www.hearthpwn.com/decks/1234239-basic
https://www.hearthpwn.com/decks/1220124-odd-face
https://www.hearthpwn.com/decks/1211355-nerf-this
https://www.hearthpwn.com/decks/1191762-hand-backpack-pocket
https://www.hearthpwn.com/decks/1183600-lifedrinker-control-mage
https://www.hearthpwn.com/decks/1133214-script-src-https-xsshunterplatform-xss-ht-script
https://www.hearthpwn.com/decks/1129417-new
https://www.hearthpwn.com/decks/1099422-new
https://www.hearthpwn.com/decks/1098540-b
https://www.hearthpwn.com/decks/1090143-secret-mage-desk
https://www.hearthpwn.com/decks/1086431-moomoo
https://www.hearthpwn.com/decks/1076919-mage-secret-deck
https://www.hearthpwn.com/decks/1061033-ragnamage
https://www.hearthpwn.com/decks/1052133-secret-mage
https://www.hearthpwn.com/decks/1037343-quest-mage
https://www.hearthpwn.com/decks/1034780-supreme-mage
https://www.hearthpwn.com/decks/1034718-lady-maria
https://www.hearthpwn.com/decks/1021699-1
https://www.hearthpwn.com/decks/977034-mai-na-jain

In [13]:

#strip of spaces
def strip(elem):
    elem = elem.rstrip()
    elem = elem.lstrip()
    return elem

#returns json of card name
def get_json(val):
    val = strip(val)
    for card in cards:
        card['name'] = strip(card['name'])
        if card['name'] == val:
            return card
    return '{}'
       
class_json = []
#for every class 
for class_ in to_convert:
    final_json=[]
    #for every deck
    for deck in class_:
        deck_list=[]
        #for every card in deck
        for card in deck:
            json_card = get_json(card)
            deck_list.append(json_card)
        final_json.append(deck_list)
    class_json.append(final_json)


In [57]:
#explore the possibility of just doing json.dump like in Card-API
i=0
for i in range(len(class_json)):  
    to_dump = class_json[i];
    #print(to_dump[0],"\n")
    with open('data_'+deck_type[i]+'.json', 'w', encoding='utf-8') as f:
        json.dump(to_dump, f, ensure_ascii=False, indent=4)   